In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy import stats
import warnings
import sys
import seaborn as sns
sys.path.append('../')
pd.options.mode.chained_assignment = None 



In [2]:
#SECONDS THRESHOLD
#used to classify multiple detections into part of a single event
#also used to tell when two detections are part of different events
#by checking the time distance between them
t = 15

#DISTANCE THRESHOLD
#used to classify an event as entering or exiting
#when two consecutive detections of the same event 
#have this distance in y position, they are utilized to predict
#the trajectory. If not then it checks the detection prior for
#the distance threshold, and continues doing so until it finds
#the last detection in the event or until it finds a distance
#of more than the threshold

t2 = 150

#ANGLE THRESHOLD
#used to generate angle ranges for classifying as exiting
#or entering

angle = 10

In [3]:
#combine all CSV's of the same data
files = open("bees/files.txt")
filenames = []
while True:
    line = files.readline().strip()
    if not line:
        break
    filenames.append("bees/" + line)

vdf = pd.concat(map(pd.read_csv, filenames), ignore_index=True)

vdf['track_endtime'] = vdf['track_endtime'].apply(lambda x: pd.to_datetime(x))
vdf['track_starttime'] = vdf['track_starttime'].apply(lambda x: pd.to_datetime(x))
vdf['timestamp'] = vdf['timestamp'].apply(lambda x: pd.to_datetime(x))
vdf['track_tagid'] = vdf['track_tagid'].apply(lambda x: str(x))
vdf = vdf.sort_values(by=['track_tagid','track_starttime']).reset_index()
vdf['timedelta'] = vdf['timestamp'].diff().shift(periods=-1)
vdf['timedelta'] = vdf['timedelta'].apply(lambda x: x.total_seconds())
vdf['separate_event'] = ((vdf['timedelta'] >= t) | (vdf['timedelta'] < 0)) 

In [4]:
break_indexes = vdf.separate_event[vdf.separate_event == True].index.tolist()
break_indexes.append(len(vdf)-1)

In [21]:
vdf['event_id'] = np.nan
first_detection = 0
for i in range(len(break_indexes)):
        
        vdf['event_id'].iloc[first_detection:break_indexes[i]+1] = i
        first_detection = break_indexes[i]+1

## RULES

In [22]:
####CLASSIFICATION AS ENTERING-EXITING OR EXITING-ENTERING

def getangle(coordinates):
    exit_min = 180 + angle
    exit_max = 360 - angle
    enter_min = angle
    enter_max = 180 - angle

    avg_x = np.cos(np.deg2rad(coordinates))
    avg_y = np.sin(np.deg2rad(coordinates))
    if avg_x == 0 and avg_y == 0:
        deg = 0
    elif avg_x == 0 and avg_y != 0:
        if avg_y > 0:
            deg = 270
        elif avg_y < 0:
            deg = 90
    else:
        # determine direction angle using arctan
        deg = np.rad2deg(np.arctan(avg_y/avg_x))
                    
        # since arctan limits are (-90,90), use coordinate directions to 
        # correct the angle to be within standard [0,360) range
        if avg_x > 0 and avg_y >= 0:
            deg = deg
        elif avg_x < 0 and avg_y >= 0:
            deg = 180 + deg
        elif avg_x < 0 and avg_y < 0:
            deg = deg + 180
        elif avg_x > 0 and avg_y < 0:
            deg = 360 + deg

    if deg >= exit_min and deg <= exit_max:
        return 'exiting'
    elif deg >= enter_min and deg <= enter_max:
        return 'entering'
    else:
        return 'unknown'

#RULE 1 ITERATE BACKWARDS AND FORWARDS DISPLACEMENT
def displacementbackforth(data):   

        coordinates = data['cy']

        #ending movement
        if len(coordinates) > 0:
            final = coordinates.iloc[-1]
            
            for k in range(len(coordinates)):
                prev = coordinates.iloc[len(coordinates)-k-1]
                dif = final - prev
                if abs(dif) >= t2:
                    if dif < 0:
                        string2 = 'exiting'
                    elif dif > 0:
                        string2 = 'entering'
                    else:
                        string2 = 'unknown'
                    break
                elif k == len(coordinates) - 1:
                    if dif < 0:
                        string2 = 'exiting'
                    elif dif > 0:
                        string2 = 'entering'
                    else:
                        string2 = 'unknown'
    
            #beginning movement
            #iterate forwards
            first = coordinates.iloc[0]
            
            for k in range(0,len(coordinates)):
                next = coordinates.iloc[k]
                dif = first - next
                if abs(dif) >= t2:
                    if dif > 0:
                        string = 'exiting'
                    elif dif < 0:
                        string = 'entering'
                    else:
                        string = 'unknown'
                    break
                elif k == len(coordinates) - 1:
                    if dif > 0:
                        string = 'exiting'
                    elif dif < 0:
                        string = 'entering'
                    else:
                        string = 'unknown'
    
            return string if string == string2 else f"{string}-{string2}"
        else:
            return 'unknown'
        

#RULE 2 FIRST AND LAST DIRECTIONAL ANGLE
    

def lastfirstangle(data):

    if len(data) > 0:
        coordinates = data['angle'].iloc[-1]
    
        string2 = getangle(coordinates)
    
        coordinates = data['angle'].iloc[0]
    
        string = getangle(coordinates)
    
        return string if string == string2 else f"{string}-{string2}"
    else:
        return 'unknown'


In [23]:
#classify all events
events = vdf['event_id'].unique()
compound_events = ['entering-exiting','exiting-entering']
countdisplacement = 0
countangle = 0

for i in events:
    detections = vdf[vdf['event_id'] == i]

    if displacementbackforth(detections) in compound_events:
        countdisplacement += 1
    if lastfirstangle(detections) in compound_events:
        countangle += 1

In [28]:
countdisplacement/len(events)

0.2619633714060654

In [31]:
countangle/len(events)

0.24753840094525403

In [26]:
len(events)

20312